In [265]:
import pandas as pd

In [266]:
## Open file
number_samples = 6
bracken_file=pd.read_csv(filepath_or_buffer='/Volumes/PiconCossio/biocomp_tools/MAUS/utils/braken_output_G1_G2.txt', 
                         delimiter='\t', skiprows=8)
bracken_file.head()

,#perc,tot_all,tot_lvl,G1_1_all,G1_1_lvl,G2_1_all,G2_1_lvl,G3_1_all,G3_1_lvl,G4_1_all,G4_1_lvl,G5_1_all,G5_1_lvl,G6_1_all,G6_1_lvl,lvl_type,taxid,name
0,0.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,U,0,unclassified
1,100.1948,1841211,0,314064,0,326812,0,231626,0,341953,0,308615,0,318141,0,R,1,root
2,86.6385,1592096,0,261690,0,233040,0,223694,0,324432,0,265100,0,284140,0,D,3,Bacteria
3,35.1509,645944,0,62650,0,30091,0,114312,0,189540,0,110498,0,138853,0,P,2375,Proteobacteria
4,28.3632,521211,0,29902,0,20580,0,92022,0,169623,0,79585,0,129499,0,C,3303,Gammaproteobacteria


In [267]:
## compute relative abundance (RelAbun)
def compute_relative_abun (df, number_samples=6):
    for i in range(1, number_samples+1):
        i = str(i)
        df[f"G{i}_1_RelAbun"] = df[f"G{i}_1_lvl"]/df[f"G{i}_1_lvl"].sum() * 100
    return df

In [268]:
bracken_file=compute_relative_abun(df=bracken_file, number_samples=number_samples)
min_abundance=1
def filter_by_abundance_conditional (df, number_samples, min_abundance, min_samples):
    ## This is intended to mimic the taxa filter by abundance conditionally of qiime2
    ## Select taxa that have at least n abundance in at least n samples 
    df = df[(df.iloc[:,number_samples*-1:] >= min_abundance).sum(axis=1) >= min_samples]
    return df

bracken_file = filter_by_abundance_conditional (df=bracken_file, 
                                                number_samples=number_samples,
                                                min_abundance=min_abundance, 
                                                min_samples=2)
bracken_file=compute_relative_abun(df=bracken_file, number_samples=number_samples)

In [269]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px


In [270]:
bracken_file_relative_abun=bracken_file.loc[:,"name":]
fig = make_subplots(rows=2, cols=1)

In [271]:
#bracken_file_relative_abun

In [272]:
sample_names=bracken_file_relative_abun.iloc[:,1:].columns
bracken_file_relative_abun["name"] = bracken_file_relative_abun["name"].apply(lambda x:x.strip())

for i in range(0, len(bracken_file_relative_abun["name"])):
      fig.add_trace(go.Bar(name=bracken_file_relative_abun.iloc[i,0].strip(),
                        x=sample_names,y=bracken_file_relative_abun.iloc[i,1:]),
                  row=1, col=1)

fig.update_layout(barmode='stack',
    autosize=True,
    width=1200,
    height=800,
    title_text="Relative abundance V3-V4 Kraken2-Bracken")
fig.show()